In [ ]:
! ls azure

preload_class.lst


In [ ]:
!pip install nltk

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-22e83cd2-59d1-4b57-9e6a-30c99c4c31e8/bin/python -m pip install --upgrade pip' command.


In [ ]:
print("#"*80)

################################################################################


In [ ]:
################################################################################
from copy import deepcopy
import re
from string import punctuation
################################################################################
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size, array, countDistinct
from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.sql import SparkSession
################################################################################
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, StringIndexer #Word2Vec, OneHotEncoder, RegexTokenizer,
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
################################################################################
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Out[212]: True

In [ ]:
t_df = sqlContext.table("train_csv") # "text dataframe"

In [ ]:
t_df.select("label").distinct().show(), t_df.count()
t_df.count()

+--------------------+
|               label|
+--------------------+
|"" Mr Flynn said....|
| it shows the two...|
|   refusing to leave|
|"" Gibson told th...|
| disrupting movem...|
| the company note...|
| according to loc...|
| 2017 at 8:10pm P...|
| the European Uni...|
| but the southwes...|
| health officer f...|
| but he saved his...|
| keep accounts or...|
| TBH. Leo got his...|
| '' as in linebacker|
| so we did it in ...|
| retraining and u...|
| and the addition...|
| but voters' feel...|
| may you rot in h...|
+--------------------+
only showing top 20 rows

Out[214]: 26568

In [ ]:
print(t_df.count())
t_df=t_df.na.drop()
print(t_df.count())

26568
19627


In [ ]:
# we have some malformed results in csv -> not all commas are removed?
t_df=t_df.filter((t_df.label == "news")|(t_df.label == "clickbait"))
t_df.count()

Out[216]: 11807

In [ ]:
t_df.filter((t_df.label == "news")).count(),t_df.filter((t_df.label == "clickbait")).count()

Out[217]: (9327, 2480)

In [ ]:
# t_df = spark.read.csv(train_path, header=True, inferSchema=True, encoding="utf-8")

# t_df.persist()

# t_df.select("label").distinct().show(), t_df.count()

# t_df.persist()

In [ ]:
# try:
#     sc.stop()
# except:
#     print("no sc to stop")

In [ ]:
min_label_count=min([ t_df.filter((t_df.label == "news")).count(),t_df.filter((t_df.label == "clickbait")).count() ]) 
min_label_count

Out[220]: 2480

In [ ]:
t_df.select('title','text','label').where(col('label')=='news').limit(20).show(), t_df.select('title','text','label').where(col('label')=='clickbait').limit(20).show()

+--------------------+--------------------+-----+
|               title|                text|label|
+--------------------+--------------------+-----+
|Mindy Finn’s Ques...|Mindy Finn wouldn...| news|
|Philippine minist...|The Philippine de...| news|
|Celebrities Offer...|The sudden death ...| news|
|Clooney used ‘Gol...|live reenactments...| news|
|Islamic State dea...|Islamic State wil...| news|
|Welcome to the re...|If you have to go...| news|
|The International...|, I want to recei...| news|
|Islamic State Fea...|The Islamic State...| news|
|Ted Cruz Counts o...|The latest polls ...| news|
|Trump vows appeal...|A defiant Preside...| news|
|At Oregon and Yal...|ANAHEIM, Calif. —...| news|
|Michael Bloomberg...|The former New Yo...| news|
|House to vote Thu...|House Republicans...| news|
|Exclusive: Teleco...|Oi SA ( ) Brazil’...| news|
|How to ditch your...|Winter is upon us...| news|
|Black militia co-...|The of a black mi...| news|
|Florida Gov. Rick...|Florida Gov. Rick...| news|


In [ ]:
print(punctuation)

@udf("string")
def strip_possessive_str(s_):
    ptrn=re.compile(r"['`’]s", flags=re.I)
    r__ = ptrn.sub(' ', s_)
    return(r__)

@udf("string")
def strip_func_str(s_):
    ptrn = re.compile('[\W_]+')
    r__ = ptrn.sub(' ', s_)
    return(r__)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:

def get_avg_len_cols(df, selection):
    """for str length in pyspark columns"""
    return(
        df \
        .select(*(length(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

from collections.abc import Iterable
def get_avg_size_cols(df, selection):
    """for arrays of str in pyspark df"""
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(size(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

In [ ]:
stemmer=SnowballStemmer("english")

lemmatizer = WordNetLemmatizer()

def stem_lem_per_word(word, pos='n'):
    sl_word = stemmer.stem(word)
    sl_word = lemmatizer.lemmatize(sl_word, pos=pos)
    return(sl_word)
    

def stem_lem_list(list_of_words, pos='n'):
    r_list = [stem_lem_per_word(w_) for w_ in list_of_words]
    r_list = [w_ for w_ in r_list if w_ not in [""]]
    return(r_list)

@udf("array<string>")
def stem_lem(arr_str):
     return stem_lem_list(arr_str)


st_cases=["writing", "write", "wrote", "writing", "written", "drag", "dragged", "dragon", ""]
[stem_lem_per_word(case) for case in st_cases]

stem_lem_list(st_cases)

Out[224]: ['write',
 'write',
 'wrote',
 'write',
 'written',
 'drag',
 'drag',
 'dragon']

In [ ]:
t_df=t_df.withColumn("s", 
                strip_func_str(strip_possessive_str(
                    concat(col('title'),col("text")))
                              ))

In [ ]:
selection = ["title", "text", "s"]

display("avg string length of columns")
get_avg_len_cols(t_df, selection)

'avg string length of columns'+-----------------+-----------------+------------------+
|            title|             text|                 s|
+-----------------+-----------------+------------------+
|71.07258406030321|3747.550859659524|3697.1343270940965|
+-----------------+-----------------+------------------+



In [ ]:
# Tokenize string to list
t_df = Tokenizer(inputCol = "s", outputCol = "s1").transform(t_df) # first text transformation is pyspark builtin tokenizer

In [ ]:
t_df.select("s1").sample(withReplacement=False, fraction=1.0, seed=news_seed).show(truncate=True)

+--------------------+
|                  s1|
+--------------------+
|[mindy, finn, que...|
|[philippine, mini...|
|[celebrities, off...|
|[clooney, used, g...|
|[islamic, state, ...|
|[welcome, to, the...|
|[the, internation...|
|[islamic, state, ...|
|[ted, cruz, count...|
|[trump, vows, app...|
|[at, oregon, and,...|
|[michael, bloombe...|
|[house, to, vote,...|
|[exclusive, telec...|
|[how, to, ditch, ...|
|[black, militia, ...|
|[florida, gov, ri...|
|[video, hillary, ...|
|[ticketmaster, wa...|
|[no, trump, immig...|
+--------------------+
only showing top 20 rows



In [ ]:
stopwords_pyspark = StopWordsRemover().getStopWords()

stopwords_from_intersection_most_common_1grams=['000',
 '1',
 '10',
 '2',
 '2017',
 '3',
 'according',
 'also',
 'another',
 'around',
 'back',
 'best',
 'breitbart', 
 'company',
 'day',
 'donald',
 'even',
 'every',
 'first',
 'get',
 'go',
 'going',
 'good',
 'home',
 'house',
 'know',
 'last',
 'life',
 'like',
 'long',
 'm',
 'made',
 'make',
 'many',
 'may',
 'million',
 'mr',
 'much',
 'need',
 'new',
 'news',
 'next',
 'old',
 'one',
 'people',
 'percent',
 'president',
 're',
 'right',
 'said',
 'say',
 'says',
 'see',
 'show',
 'since',
 'state',
 'still',
 'take',
 'think',
 'three',
 'time',
 'times',
 'told',
 'trump',
 'twitter',
 'two',
 'u',
 'us',
 'want',
 'way',
 'week',
 'well',
 'women',
 'work',
 'world',
 'year',
 'years',
 ]

In [ ]:
t_df = StopWordsRemover(inputCol = "s1", outputCol="s2", stopWords=stopwords_pyspark).transform(t_df)

In [ ]:
display("difference in avg word array len after StopWordsRemover.transform")
get_avg_size_cols(t_df, ['s1','s2'])

'difference in avg word array len after StopWordsRemover.transform'+-----------------+-----------------+
|               s1|               s2|
+-----------------+-----------------+
|647.0234606589312|372.6026086220039|
+-----------------+-----------------+



In [ ]:
t_df.sample(False, 0.1, seed=999).select("s2").limit(5).show(truncate=512)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
gramrange=range(1,10)

ngram_pipeline=Pipeline(stages=[NGram(n=i, inputCol="s2", outputCol= str(i)+"gram") for i in gramrange])
m = ngram_pipeline.fit(t_df)
t_df = m.transform(t_df)
t_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- s1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- s2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 2gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 3gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 4gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 5gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 6gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 7gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 8gram: array (nullable = true)
 |    |-- element: string (cont

In [ ]:
# grams=['{}gram'.format(str(n)) for n in range(1,4)]
grams=['{}gram'.format(str(n)) for n in gramrange]
["label"]+grams

Out[234]: ['label',
 '1gram',
 '2gram',
 '3gram',
 '4gram',
 '5gram',
 '6gram',
 '7gram',
 '8gram',
 '9gram']

In [ ]:
t_df.select('label', '1gram', '2gram', '3gram').sample(False, 0.1, seed=91).select("2gram").limit(3).show(truncate=50)
# t_df.select('label').sample(False, 0.1, seed=91).limit(3).show(truncate=True)

+--------------------------------------------------+
|                                             2gram|
+--------------------------------------------------+
|[oregon yale, yale march, march madness, madnes...|
|[secrets mariana, mariana trench, trench caught...|
|[parting words, words sec, sec white, white cal...|
+--------------------------------------------------+



In [ ]:
indexer = StringIndexer(inputCol="label", outputCol="label_index")
t_df = indexer.fit(t_df).transform(t_df)

In [ ]:
t_df.select('label',"label_index", '1gram').sample(False, 0.1, seed=91).limit(3).show(truncate=True)

t_df.select("label_index").distinct().show(),t_df.count()

+-----+-----------+--------------------+
|label|label_index|               1gram|
+-----+-----------+--------------------+
| news|        0.0|[oregon, yale, ma...|
| news|        0.0|[secrets, mariana...|
| news|        0.0|[parting, words, ...|
+-----+-----------+--------------------+

+-----------+
|label_index|
+-----------+
|        0.0|
|        1.0|
+-----------+

Out[237]: (None, 11807)

In [ ]:
t_df.groupBy('label', 'label_index').count().show()

+---------+-----------+-----+
|    label|label_index|count|
+---------+-----------+-----+
|clickbait|        1.0| 2480|
|     news|        0.0| 9327|
+---------+-----------+-----+



In [ ]:
train, test = t_df.randomSplit([.8,.2], seed = 999)

In [ ]:
pipeline = Pipeline(stages=[
                    CountVectorizer(inputCol=c__ , outputCol=c__+"_cv", minDF=2.0) for c__ in grams] +
                    [NaiveBayes(modelType= "multinomial", labelCol="label_index",
                                featuresCol = c__ + "_cv",
                                rawPredictionCol="raw_pred_" + c__,
                                probabilityCol="prob_"+c__,
                                predictionCol="pred_"+c__) for c__ in grams]
                   )

In [ ]:
# fit and predict
pipeline_model = pipeline.fit(train)

pred = pipeline_model.transform(test)

In [ ]:
evaluation=[MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = f"pred_{g}", 
                                              metricName="accuracy") for g in grams]

evaluation = [(e, e.evaluate(pred) )for e in evaluation]

evaluation

Out[242]: [(MulticlassClassificationEvaluator_d9d20a9ef751, 0.7352101183190535),
 (MulticlassClassificationEvaluator_d5386c2e19a0, 0.8078335373317014),
 (MulticlassClassificationEvaluator_57ced4b289dc, 0.8057935536515708),
 (MulticlassClassificationEvaluator_b0def8d2b6b2, 0.817625458996328),
 (MulticlassClassificationEvaluator_54c4bed63ae2, 0.8257853937168502),
 (MulticlassClassificationEvaluator_294a0fac7a52, 0.8266013871889025),
 (MulticlassClassificationEvaluator_62e9d509eb63, 0.8270093839249286),
 (MulticlassClassificationEvaluator_228f9e732f1e, 0.8274173806609547),
 (MulticlassClassificationEvaluator_b59da7664345, 0.8278253773969808)]

In [ ]:
[print(f"{g} accuracy : {e[1]}") for e,g in zip(evaluation, grams)]

1gram accuracy : 0.7352101183190535
2gram accuracy : 0.8078335373317014
3gram accuracy : 0.8057935536515708
4gram accuracy : 0.817625458996328
5gram accuracy : 0.8257853937168502
6gram accuracy : 0.8266013871889025
7gram accuracy : 0.8270093839249286
8gram accuracy : 0.8274173806609547
9gram accuracy : 0.8278253773969808
Out[243]: [None, None, None, None, None, None, None, None, None]